In [42]:
import sys
sys.path.insert(0, '/home/jindal/notebooks/jindal/NER/')
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess
import fastText

In [32]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]


def createMatrices(sentences, label2Idx, case2Idx,char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}

        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []
        caseIndices = []
        charIndices = []
        labelIndices = []
        
        for word,char,label in sentence:  
            charIdx = []
            for x in char:
                if x in char2Idx.keys():
                    charIdx.append(char2Idx[x])
                else:
                    charIdx.append(char2Idx['UNKNOWN'])
            #Get the label and map to int            
            wordIndices.append(word)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, charIndices, labelIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences



In [3]:
epochs = 80
trainable = True

In [4]:
from keras_contrib.layers import CRF


In [5]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [6]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [7]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [112]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [39]:
trainSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [113]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12152
2867
3005


In [44]:
print(trainSentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [54]:
labelSet = set()
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)

In [47]:
print(labelSet)

{'I-ORGderiv', 'I-PERpart', 'I-LOCpart', 'I-ORGpart', 'B-PER', 'I-PERderiv', 'B-OTH', 'I-LOCderiv', 'B-OTHpart', 'B-OTHderiv', 'B-ORGpart', 'I-PER', 'B-LOC', 'I-OTHpart', 'O', 'B-LOCderiv', 'I-OTHderiv', 'I-OTH', 'B-ORGderiv', 'B-PERpart', 'I-ORG', 'B-PERderiv', 'B-ORG', 'I-LOC', 'B-LOCpart'}


In [48]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [49]:
print(label2Idx)

{'B-LOC': 12, 'B-PERderiv': 21, 'I-OTHpart': 13, 'O': 14, 'B-LOCderiv': 15, 'I-ORGderiv': 0, 'I-OTH': 17, 'I-PERpart': 1, 'I-LOCpart': 2, 'I-OTHderiv': 16, 'B-PERpart': 19, 'I-ORGpart': 3, 'B-PER': 4, 'I-ORG': 20, 'B-ORGderiv': 18, 'B-OTH': 6, 'I-PERderiv': 5, 'B-ORG': 22, 'I-LOC': 23, 'I-LOCderiv': 7, 'B-OTHpart': 8, 'B-LOCpart': 24, 'B-OTHderiv': 9, 'B-ORGpart': 10, 'I-PER': 11}


In [50]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [51]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'mainly_numeric': 5, 'numeric': 0, 'allUpper': 2, 'contains_digit': 6, 'initialUpper': 3, 'other': 4, 'PADDING_TOKEN': 7, 'allLower': 1}


In [17]:
print(type(trainSentences))

<class 'list'>


In [52]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [55]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'İ': 317, 'ź': 0, ')': 1, 'å': 2, 'ð': 3, ',': 4, 'ĩ': 207, '妃': 6, 'ø': 8, 'κ': 303, 'ɨ': 10, '\x99': 173, '⋅': 12, 'á': 273, '傳': 15, 'd': 14, '鷹': 18, '冲': 17, '柯': 188, 'q': 291, 'к': 19, '守': 318, 't': 20, '´': 21, '造': 22, 'D': 23, 'η': 5, 'ä': 25, '’': 26, 'є': 28, 'ý': 29, 'Π': 30, 'ῦ': 31, '×': 34, '=': 33, '公': 35, 'ǒ': 36, 'N': 279, "'": 37, 'n': 38, '(': 40, '寝': 114, 'G': 41, '樓': 42, 'Е': 9, 'C': 43, 'y': 44, 'ρ': 46, '博': 48, '士': 49, '0': 50, 'β': 51, 'Λ': 52, 'ι': 53, 'È': 11, 'l': 55, '鶴': 56, 'ό': 299, '학': 57, 'ē': 61, '%': 59, '6': 60, 'ḳ': 63, 'ę': 241, '貴': 64, 'и': 287, 'ś': 65, 'ğ': 66, '⊃': 264, 'ж': 67, 'ñ': 68, 'ـ': 226, '9': 120, '별': 69, '¹': 70, 'р': 71, 'ệ': 73, '루': 294, 'И': 74, 'б': 247, 'Þ': 75, 'ł': 77, 'ň': 304, '*': 78, 'ť': 84, '\x92': 81, 'õ': 83, 'Å': 315, 'Î': 86, 'H': 87, 'ї': 90, 'П': 89, 'ά': 13, 'Ä': 91, 'ā': 155, '章': 92, '„': 93, 'ć': 94, 'Č': 95, 'é': 96, 'P': 97, 'ó': 100, 'オ': 102, 'è': 316, '1': 103, '\x96': 104, '\xad': 105, 'г': 1

In [56]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [43]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()

In [61]:
def train_generator(dataset: 'list of training/dev sentences', batch_size:'specify the batch size'):
    global line_number
    while True:
        word_embeddings = []
        case_embeddings = []
        char_embeddings = []
        output_labels = []
        total_lines = len(dataset)
        for i in range(batch_size):            
            index = line_number%total_lines +1
            line_number+=1
            sentence = dataset[index]
            temp_casing = []
            temp_char=[]
            temp_word=[]
            temp_output=[]
            for word in sentence:
                word, label = word
                casing =getCasing(word, case2Idx)
                temp_casing.append(casing)
                temp_char2=[]
                for char in word:
                    if char in char2Idx.keys():
                        temp_char2.append(char2Idx[char])
                    else:
                        temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                temp_char2 = np.array(temp_char2)
                temp_char.append(temp_char2)
                word_vector = ft.get_word_vector(word.lower())
                temp_word.append(word_vector)
                temp_output.append(label2Idx[label])
            temp_char = pad_sequences(temp_char, 52)
            word_embeddings.append(temp_word)
            case_embeddings.append(temp_casing)
            char_embeddings.append(temp_char)
            output_labels.append(temp_output)
        yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))
            
    

In [79]:
line_number=0
for inp, out in train_generator(trainSentences,1):
    word, case, char = inp
    print(word.shape)
    print(case.shape)
    print(char.shape)
    print(out.shape)
    print("********")

(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 7, 300)
(1, 7)
(1, 7, 52)
(1, 7)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 44, 300)
(1, 44)
(1, 44, 52)
(1, 44)
********
(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 32, 300)
(1, 32)
(1, 32, 52)
(1, 32)
********
(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27)
********
(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6)
********
(1, 15, 300)
(1, 15)
(1, 15, 52)
(1, 15)
********
(1, 9, 300)
(1, 

(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 32, 300)
(1, 32)
(1, 32, 52)
(1, 32)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27)
********
(1, 29, 300)
(1, 29)
(1, 29, 52)
(1, 29)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26)
********
(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 15, 300)
(1, 15)
(1, 15, 52)
(1, 15)
********
(1, 29, 300)
(1, 29)
(1, 29, 52)
(1, 29)
********
(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 17, 300)

(1, 17, 52)
(1, 17)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 23, 300)
(1, 23)
(1, 23, 52)
(1, 23)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 35, 300)
(1, 35)
(1, 35, 52)
(1, 35)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 25, 300)
(1, 25)
(1, 25, 52)
(1, 25)
********
(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33)
********
(1, 17, 300)
(1, 17)
(1, 17, 52)
(1, 17)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 

(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 41, 300)
(1, 41)
(1, 41, 52)
(1, 41)
********
(1, 38, 300)
(1, 38)
(1, 38, 52)
(1, 38)
********
(1, 25, 300)
(1, 25)
(1, 25, 52)
(1, 25)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 33, 300)
(1, 33)
(1, 33, 52)
(1, 33)
********
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19)
********
(1, 29, 300)
(1, 29)
(1, 29, 52)
(1, 29)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 26, 300)
(1, 26)
(1, 26, 52)
(1, 26)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********


(1, 25, 300)
(1, 25)
(1, 25, 52)
(1, 25)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20)
********
(1, 15, 300)
(1, 15)
(1, 15, 52)
(1, 15)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 30, 300)
(1, 30)
(1, 30, 52)
(1, 30)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 42, 300)
(1, 42)
(1, 42, 52)
(1, 42)
********
(1, 17, 300)
(1, 17)
(1, 17, 52)
(1, 17)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 32, 300)
(1, 32)
(1, 32, 52)
(1, 32)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 35, 300)
(1, 35)
(1, 35, 52)
(1, 35)
********
(1, 39, 300)
(1, 39)
(1, 39, 52)
(1, 39)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 27, 300)
(1, 27)
(1, 27, 52)
(1, 27)
********
(1, 19, 

...

(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 40, 300)
(1, 40)
(1, 40, 52)
(1, 40)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 7, 300)
(1, 7)
(1, 7, 52)
(1, 7)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 23, 300)
(1, 23)
(1, 23, 52)
(1, 23)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 23, 300)
(1, 23)
(1, 23, 52)
(1, 23)
********
(1, 34, 300)
(1, 34)
(1, 34, 52)
(1, 34)
********
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 15, 

(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 20, 300)
(1, 20)
(1, 20, 52)
(1, 20)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 31, 300)
(1, 31)
(1, 31, 52)
(1, 31)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 24, 

(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 11, 300)
(1, 11)
(1, 11, 52)
(1, 11)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 5, 300)
(1, 5)
(1, 5, 52)
(1, 5)
********
(1, 16, 300)
(1, 16)
(1, 16, 52)
(1, 16)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 34, 300)
(1, 34)
(1, 34, 52)
(1, 34)
********
(1, 34, 300)
(1, 34)
(1, 34, 52)
(1, 34)
********
(1, 24, 300)
(1, 24)
(1, 24, 52)
(1, 24)
********
(1, 7, 300)
(1, 7)
(1, 7, 52)
(1, 7)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 10, 300)
(1, 10)
(1, 10, 52)
(1, 10)
********
(1, 6, 300)
(1, 6)
(1, 6, 52)
(1, 6)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 30, 300)
(1, 30)
(1, 30,

(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 9, 300)
(1, 9)
(1, 9, 52)
(1, 9)
********
(1, 13, 300)
(1, 13)
(1, 13, 52)
(1, 13)
********
(1, 12, 300)
(1, 12)
(1, 12, 52)
(1, 12)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 19, 300)
(1, 19)
(1, 19, 52)
(1, 19)
********
(1, 23, 300)
(1, 23)
(1, 23, 52)
(1, 23)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 32, 300)
(1, 32)
(1, 32, 52)
(1, 32)
********
(1, 25, 300)
(1, 25)
(1, 25, 52)
(1, 25)
********
(1, 21, 300)
(1, 21)
(1, 21, 52)
(1, 21)
********
(1, 23, 300)
(1, 23)
(1, 23, 52)
(1, 23)
********
(1, 18, 300)
(1, 18)
(1, 18, 52)
(1, 18)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 28, 300)
(1, 28)
(1, 28, 52)
(1, 28)
********
(1, 8, 300)
(1, 8)
(1, 8, 52)
(1, 8)
********
(1, 22, 300)
(1, 22)
(1, 22, 52)
(1, 22)
********
(1, 14, 300)
(1, 14)
(1, 14, 52)
(1, 14)
********
(1, 15, 300)
(1,

KeyboardInterrupt: 

In [70]:
idx2Label = {v: k for k, v in label2Idx.items()}


In [71]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [75]:
words_input = Input(shape=(None,300),dtype='float32',name='words_input')
# words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes[:1]:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words_input, casing, conv_blocks[0]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_33 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
time_distr

In [77]:
model.fit_generator(train_generator(trainSentences, 2), epochs=10, steps_per_epoch= len(trainSentences)//2,  )

Epoch 1/10


ValueError: Error when checking input: expected words_input to have 3 dimensions, but got array with shape (2, 1)

In [ ]:
for x in range(15,20):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
#             pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
            predLabels = np.concatenate(predLabels).ravel()
            correctLabels = np.concatenate(correctLabels).ravel()
            f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
#             print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
            print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_conll.h5')
                maxf1 = f1_dev
    #         predLabels, correctLabels = tag_dataset(dev_batch)        
        #     with open("dropout=0.75.txt",'w') as f:
    #         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
        #     with open('dropout=0.75.txt') as f:
        #         x = str(epoch)+ " "+f1_dev
        #         f.write(x)
        #         f.write('\n')
    #         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))


    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    model.load_weights('german_ner_conll.h5')
    predLabels, correctLabels = tag_dataset(test_batch)        
    # pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
    # print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

    id = 1
    file_name='test_conll'+str(x)+'.txt'
    input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
    f = open(input_file_name,'wb')
    for sentence_number, batch in enumerate(test_batch):
        for word_number, wordid in enumerate(batch[0]):
            word = id2word[wordid]
            true_label = correctLabels[sentence_number][word_number]
            true_label = idx2Label[true_label]
            pred_label = predLabels[sentence_number][word_number]
            pred_label = idx2Label[pred_label]

#             string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
            string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
            string = string.encode()
            f.write(string)
            id+=1

    # command = "perl nereval.perl"
    # directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
    directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
    command = "perl conlleval"
    
    output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_conll'+str(x)+'.txt'
    output_file=open(output_file_name,'w')
    input_file = open(input_file_name)
    import subprocess
    process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_conll)
    out, err = process.communicate()
#     print(out)
    print('******************************************************************************')

In [76]:
import os
os.getcwd()

'/home/jindal/notebooks/Resources/GermEVAL'